# velMANNTrajectoryGenerator
This notebook contains the code showing how to use velMANNTrajectoryGenerator to generate a walking trajectory for the ergoCub humanoid robot. 

If you are interested in some details of the algorithm, please refer to the paper: [Trajectory Generation with Physics-Informed Learning and Drift Mitigation](https://github.com/ami-iit/paper_delia_2024_ral_physics-informed_trajectory_generation).

Differently from the `velMANNAutoregressive`, `velMANNTrajectoryGenerator` class can be used to plan a trajectory that can be fed into an MPC. Indeed it is possible to sample the contact and extract a `ContactPhaseList` that can be used to initialize the MPC. Moreover the `velMANNTrajectoryGenerator` can be reset to any point of the trajectory, allowing to replan at will.

## Import all the required packages
In this section we import all the required packages. In particular, we need `iDynTree` to correctly visualize the robot, `numpy` to perform some basic operations, `manifpy` to perform some basic operations on manifolds, `resolve_robotics_uri_py` to correctly locate the `ergoCub` model. Finally `bipedal_locomotion_framework` implements the MANN network.

In [ ]:
from idyntree.visualize import MeshcatVisualizer
import idyntree.bindings as idyn
from pathlib import Path
import numpy as np
import manifpy as manif
import bipedal_locomotion_framework as blf
import resolve_robotics_uri_py

## Prepare the ingredients
To correctly run the notebook, we need to prepare the ingredients. In particular, we need to:
- Get the network model: In this case we use the model trained on the ergoCub robot. The model is available [here](https://huggingface.co/ami-iit/mann/resolve/main/ergocubSN000_26j_49e_velbased_mann.onnx).
- Load the configuration file: The configuration file contains all the parameters needed to correctly run the network and generate a proper set of inputs for the network. In detail we have two configuration files:
    - `config_mann.toml`: This file contains the parameters needed to correctly load and run the network.
    - `config_joypad.toml`: This file contains the parameters needed to correctly generate the input for the network. In particular, it contains the parameters needed to correctly map a joypad input to the network input.
- Load the robot model: In this case we use the `ergoCub` model. We load the model only to correctly visualize the robot.

### Get the network model
What we need to do is to download the network model and save it in the `config` folder. The model is available [here](https://huggingface.co/ami-iit/mann/resolve/main/ergocubSN000_26j_49e_velbased_mann.onnx). 
Moreover we load the parameters needed to correctly run the network.

In [ ]:
# We use urllib to download the onnx model from huggingface
import urllib.request
import os
url = "https://huggingface.co/ami-iit/mann/resolve/main/ergocubSN000_26j_49e_velbased_mann.onnx"
urllib.request.urlretrieve(url, "./config/ergocubSN000_26j_49e.onnx")

# Get the configuration files
config_path = Path("__file__").parent / "config" / "config_mann.toml"
params_network = blf.parameters_handler.TomlParametersHandler()
params_network.set_from_file(str(config_path))

joypad_config_path = Path("__file__").parent / "config" / "config_joypad.toml"
params_joypad = blf.parameters_handler.TomlParametersHandler()
params_joypad.set_from_file(str(joypad_config_path))

### Load ergoCub model
We load the `ergoCub` model to correctly visualize the robot.
The model is loaded using `iDynTree` and `resolve_robotics_uri_py` to correctly locate the model. Please notice that the model is loaded specifying the same joint order used to train the network.
Moreover we load a set of boxes to correctly visualize the contact points.

In [ ]:
# Get the path of the robot model
robot_model_path = str(resolve_robotics_uri_py.resolve_robotics_uri("package://ergoCub/robots/ergoCubSN001/model.urdf"))
ml = idyn.ModelLoader()
ml.loadReducedModelFromFile(robot_model_path, params_network.get_parameter_vector_string("joints_list"))
viz = MeshcatVisualizer()
viz.load_model(ml.model(), "real", 0.3)
viz.load_model(ml.model(), "ghost", 0.8)
viz.load_box(0.2, 0.1, 0.001, "left1", [219/255.0, 68/255.0, 55/255.0, 1])
viz.load_box(0.2, 0.1, 0.001, "left2", [219/255.0, 68/255.0, 55/255.0, 1])
viz.load_box(0.2, 0.1, 0.001, "right1", [219/255.0, 68/255.0, 55/255.0, 1])
viz.load_box(0.2, 0.1, 0.001, "right2", [219/255.0, 68/255.0, 55/255.0, 1])

## Network initialization
In this section we initialize the network. In particular we first instantiate the network and then we load the parameters. The parameters are loaded from the `config_mann.toml` file.

In [ ]:
# Create the trajectory generator
mann_trajectory_generator = blf.ml.velMANNTrajectoryGenerator()
assert mann_trajectory_generator.set_robot_model(ml.model())
assert mann_trajectory_generator.initialize(params_network)

# Create the input builder
input_builder = blf.ml.velMANNAutoregressiveInputBuilder()
assert input_builder.initialize(params_joypad)

# Create the input
mann_trajectory_generator_input = blf.ml.velMANNTrajectoryGeneratorInput()
mann_trajectory_generator_input.merge_point_index = 0

We now need to reset the network to the initial state. In particular, we need to reset  the joint state and the base pose in a given configuration already seen by the network during traning.

In [ ]:
# Initial joint positions configuration. The serialization is specified in the config file
joint_positions = params_network.get_parameter_vector_float("initial_joints_configuration")

# Initial base pose. This pose makes the robot stand on the ground with the feet flat
base_pose = manif.SE3.Identity()
initial_base_height = params_network.get_parameter_float("initial_base_height")
quat = params_network.get_parameter_vector_float("initial_base_quaternion")
quat = quat / np.linalg.norm(quat) # Normalize the quaternion
base_pose = manif.SE3([0, 0, initial_base_height], quat)

We finally ask the input builder to generate an input that mimics a joypad input. In particular, we ask the input builder to generate an input that mimics a joypad input corresponding to a forward walking, i.e. a joypad input with the axes moved to the following values:
- `left_stick_x`: 1.0
- `left_stick_y`: 0.0
- `right_stick_x`: 1.0
- `right_stick_y`: 0.0

In [ ]:
input_builder_input = blf.ml.velMANNDirectionalInput()
input_builder_input.motion_direction = np.array([1, 0])
input_builder_input.base_direction = np.array([1, 0])

## Network execution
We now visualize the robot in the viewer and we run the network for 50 steps. In particular, we run the trajectory generator for 50 steps and we visualize the result in the viewer. For each step we show:
- The entire future trajectory of the robot.
- The sampled contacts.
- The robot at the merge point. The merge point is the point from which the network starts to generate the trajectory at the next iteration.

In [ ]:
viz.jupyter_cell()

In [ ]:
# Reset the trajectory generator
far_position = np.array([1e5, 1e5, 1e5])
identity = np.eye(3)
mann_trajectory_generator.set_initial_state(joint_positions, base_pose)

for _ in range(50):

    # Set the input to the builder and
    input_builder.set_input(input_builder_input)
    assert input_builder.advance()
    assert input_builder.is_output_valid()

    mann_trajectory_generator_input.desired_future_base_trajectory = input_builder.get_output().desired_future_base_trajectory
    mann_trajectory_generator_input.desired_future_base_velocities = input_builder.get_output().desired_future_base_velocities
    mann_trajectory_generator_input.desired_future_base_directions = input_builder.get_output().desired_future_base_directions

    # Set the input to the trajectory generator
    mann_trajectory_generator.set_input(mann_trajectory_generator_input)
    assert mann_trajectory_generator.advance()
    assert mann_trajectory_generator.is_output_valid()

    # Get the output of the trajectory generator and update the visualization
    mann_output = mann_trajectory_generator.get_output()

    # Move the feet to a far position
    for i in range(1,3):
        viz.set_primitive_geometry_transform(far_position, identity, "left" + str(i))
        viz.set_primitive_geometry_transform(far_position, identity, "right" + str(i))

    # Show the contact sampled by the trajectory generator
    contact_map = mann_output.phase_list.lists()
    i = 1
    for contact in contact_map["left_foot"]:
        viz.set_primitive_geometry_transform(contact.pose.translation(), contact.pose.rotation(), "left" + str(i))
        i += 1

    i = 1
    for contact in contact_map["right_foot"]:
        viz.set_primitive_geometry_transform(contact.pose.translation(), contact.pose.rotation(), "right" + str(i))
        i += 1

    # Show the trajectory
    for i in range(len(mann_output.joint_positions)):
        viz.set_multibody_system_state(mann_output.base_poses[i].translation(),
                                       mann_output.base_poses[i].rotation(),
                                       mann_output.joint_positions[i],
                                       "real")

        # Show the robot at the merge point
        if i == mann_trajectory_generator_input.merge_point_index:
            viz.set_multibody_system_state(mann_output.base_poses[i].translation(),
                                           mann_output.base_poses[i].rotation(),
                                           mann_output.joint_positions[i],
                                           "ghost")

    mann_trajectory_generator_input.merge_point_index = 5